In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/projects/cft_vlm/finetune')

In [2]:
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLProcessor
import torch


processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", use_fast=True)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [2]:
import datasets

ds = datasets.load_from_disk("/projects/cft_vlm/datasets/path_vqa/data/dataset")['test']


In [5]:
from qwenvl.data.module import DatasetWrapper

dw = DatasetWrapper(ds, [[i] for i in range(len(ds))])
dw[1:5]

[[{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=CMYK size=335x400>,
   'question': 'how are the histone subunits charged?',
   'answer': 'positively charged'}],
 [{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=CMYK size=315x337>,
   'question': 'what is showing increased eosinophilia of cytoplasm, and swelling of occasional cells?',
   'answer': 'early (reversible) ischemic injury'}],
 [{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=CMYK size=492x324>,
   'question': 'does mycobacterium avium infection in a duodenal biopsy from a patient with aids show massive intracellular macrophage infection with acid-fast organisms filamentous and pink in this acid-fast stain preparation?',
   'answer': 'yes'}],
 [{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1076x421>,
   'question': 'what shows branching papillae having flbrovascular stalk covered by a single layer of cuboidal cells having ground-glass nuclei?',
   'answer': 'microscopy'}]]

In [ ]:
import shutil


i = 0
def dummy(example):
  example['i'] = 0
  return example
  
ds2 = ds.map(dummy, num_proc=24).remove_columns(["i"])
shutil.rmtree("/projects/cft_vlm/datasets/open_pmc/data/dataset/test")


PermissionError: Tried to overwrite /fs01/projects/cft_vlm/datasets/open_pmc/data/dataset/test but a dataset can't overwrite itself.

In [19]:
ds2.save_to_disk("/projects/cft_vlm/datasets/open_pmc/data/dataset/test_tmp")
shutil.move("/projects/cft_vlm/datasets/open_pmc/data/dataset/test_tmp", "/projects/cft_vlm/datasets/open_pmc/data/dataset/test")

Saving the dataset (0/8 shards):   0%|          | 0/217980 [00:00<?, ? examples/s]

'/projects/cft_vlm/datasets/open_pmc/data/dataset/test'

In [20]:
!ls /projects/cft_vlm/datasets/open_pmc/data/dataset/test

data-00000-of-00008.arrow  data-00004-of-00008.arrow  dataset_info.json
data-00001-of-00008.arrow  data-00005-of-00008.arrow  state.json
data-00002-of-00008.arrow  data-00006-of-00008.arrow
data-00003-of-00008.arrow  data-00007-of-00008.arrow


In [6]:
from qwenvl.data.input_processor import InputProcessor


ip = InputProcessor(processor)


In [8]:
def decode_output(out, processor):
  output_ids = out.logits.argmax(dim=-1)
  output_texts = processor.batch_decode(output_ids, skip_special_tokens=False)
  return output_texts

In [12]:
processor.decode([220,  ])

' '

In [11]:
processor.decode([293,   ])

' b'

In [5]:
from qwenvl.argument import ProcessingArguments

args = ProcessingArguments()
print(**args.__dict__)

TypeError: print() takes at most 4 keyword arguments (11 given)

In [ ]:
from pathlib import Path
vid_dir = Path('/projects/cft_vlm/datasets/openbiomedvid/data/vid_processed')

conversations = [[
  {"role": "user",
   "content": [
    {'text': "Describe the video.\n"},
    # {'video': vid_dir / '00cD-55fnaw_2590_3016.mp4'}
    ]},
  {"role": "assistant",
   "content": [
    {'text': "This video describes.\n"}
    ]}],
  [{"role": "user",
   "content": [
     {'text': "what is in this video?"},
    #  {'video': vid_dir / '01KY3GQKBzg_14_158.mp4'}
   ]},
  {"role": "assistant",
   "content": [{'text': "this video shows a person doing something interesting.\n"}]
  }
]]
texts = processor.apply_chat_template(conversations, add_generation_prompt=True)
texts_2 = processor.apply_chat_template([m for c in conversations for m in c], add_generation_prompt=True)
texts_lengths = [len(processor.tokenizer.encode(t)) for t in texts]
feat = processor(text=texts_2, return_tensors='pt', padding=True)
attn_mask = torch.zeros_like(feat.input_ids)
attn_mask[0, :texts_lengths[0]] = 1
attn_mask[1, -texts_lengths[1]:] = 1
feat.attention_mask = attn_mask
out = model(**feat)
decode_output(out, processor)

['<|im_end|><|im_end|>1服务中心 to multi AI<|im_end|><|im_end|>\n<|im_start|>\n<|im_end|>I<|im_end|> process<|im_end|><|im_end|>\n<|im_start|>\n:I is is a<|im_end|>\n<|endoftext|>user<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>\n<|im_start|>\n:I video is a person riding a..<|im_end|>\n<|im_start|>user:This']

In [15]:
feat.attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [13]:
texts_2 = processor.apply_chat_template([m for c in conversations for m in c], add_generation_prompt=True)
feat_2 = processor(text=texts_2, return_tensors='pt', padding=True)
out_2 = model(**feat_2)
decode_output(out_2, processor)

['<|im_end|><|im_end|>1服务中心 to multi AI<|im_end|><|im_end|>\n<|im_start|>\n<|im_end|>I<|im_end|> process<|im_end|><|im_end|>\n<|im_start|>\n:I is is a<|im_end|>\n<|endoftext|>user<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>\n<|im_start|>\n:I video is a person riding a..<|im_end|>\n<|im_start|>user:This']

In [10]:
feat_2.attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [12]:
feat.attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
data_di

In [4]:
import torch
import torch.nn.functional as F
from typing import List, Literal


def pad_and_stack_tensors(
    tensors: List[torch.Tensor],
    target_length: int,
    padding_value: int = 0,
    padding_side: Literal["right", "left"] = "right",
    truncation_side: Literal["right", "left"] = "right",
    **kwargs,
) -> torch.Tensor:
  """
  Pads or truncates a list of 1D tensors to a target length and stacks them.

  This function processes a list of 1D tensors to ensure they all have the
  same specified `target_length`. Tensors shorter than the target are padded,
  and tensors longer than the target are truncated. The processed tensors are
  then stacked into a single 2D tensor.

  Args:
      tensors (List[torch.Tensor]):
          A list of 1D PyTorch tensors to process.
      target_length (int):
          The desired final length for each tensor.
      padding_value (int, optional):
          The value to use for padding. Defaults to 0.
      padding_side (Literal["right", "left"], optional):
          The side to add padding to if a tensor is shorter than
          `target_length`. Defaults to "right".
      truncation_side (Literal["right", "left"], optional):
          The side to truncate from if a tensor is longer than
          `target_length`. Defaults to "right".

  Returns:
      torch.Tensor:
          A 2D tensor of shape (len(tensors), target_length) containing the
          processed and stacked tensors. If the input list is empty, an empty
          tensor of shape (0, target_length) is returned.

  Raises:
      ValueError: If `padding_side` or `truncation_side` are not "left" or "right".
  """
  # --- 1. Input Validation ---
  if padding_side not in ["right", "left"]:
    raise ValueError(
        f"padding_side must be 'right' or 'left', but got '{padding_side}'")
  if truncation_side not in ["right", "left"]:
    raise ValueError(
        f"truncation_side must be 'right' or 'left', but got '{truncation_side}'")

  # --- 2. Handle Edge Case: Empty Input List ---
  if not tensors:
    # Return an empty tensor with the correct shape.
    # Assuming torch.long for token IDs, but could be adapted.
    return torch.empty((0, target_length), dtype=torch.long)
  attention_masks = [] # <-- ADDED
  processed_tensors = []
  # Use the device of the first tensor for consistency
  device = tensors[0].device

  # --- 3. Process each tensor ---
  for tensor in tensors:
    current_len = tensor.size(0)

    # Ensure tensor is 1D
    if tensor.dim() != 1:
      raise ValueError(
          f"All tensors in the input list must be 1D, but found a tensor with shape {tensor.shape}")

    if current_len >= target_length:
      if truncation_side == 'right':
        processed_tensor = tensor[:target_length]
      else:
        processed_tensor = tensor[-target_length:]
      mask = torch.ones(target_length, dtype=torch.long, device=device)
    
    elif current_len < target_length:
      pad_len = target_length - current_len
      if padding_side == 'right':
        padding = (0, pad_len)
      else:
        padding = (pad_len, 0)
    
      processed_tensor = F.pad(tensor, padding, "constant", padding_value)
      mask = torch.ones(current_len, dtype=torch.long, device=device)
      mask = F.pad(mask, padding, "constant", 0)
        
    processed_tensors.append(processed_tensor)
    attention_masks.append(mask)
    
  stacked_tensors = torch.stack(processed_tensors)
  stacked_masks = torch.stack(attention_masks)

  return stacked_tensors, stacked_masks # <-- MODIFIED

In [6]:
for k in 'labels', 'input_ids', 'attention_mask':
  print(k)
  print('----', features[k].shape, data_dict[k].shape, torch.equal(features[k], data_dict[k]))

labels
---- torch.Size([1, 26878]) torch.Size([1, 26878]) True
input_ids
---- torch.Size([1, 26878]) torch.Size([1, 26878]) True
attention_mask
---- torch.Size([1, 26878]) torch.Size([1, 26878]) True


In [7]:
for k in 'image_grid_thw', 'video_grid_thw', 'second_per_grid_ts', 'pixel_values_videos', 'pixel_values_images':
  print(k)
  fk = features.get(k, None)
  dk = data_dict.get(k, None)
  if fk is None or dk is None:
    print('    ', fk is not None, dk is not None)
  else:
    print('    ', fk.shape if fk is not None else None, dk.shape if dk is not None else None, torch.equal(fk, dk))

image_grid_thw
     False False
video_grid_thw
     torch.Size([2, 3]) torch.Size([2, 3]) True
second_per_grid_ts
     torch.Size([2]) torch.Size([2]) True
pixel_values_videos
     torch.Size([107280, 1176]) torch.Size([107280, 1176]) True
pixel_values_images
     False False


In [8]:
data_dict.second_per_grid_ts

tensor([2.0262, 2.0000])